# Stable Diffusion WebUI 带lora训练
## 阿里云DSW 一键脚本 By bilibili@十字鱼
### 参考感谢——秋葉aaaki、minicacas、青龙圣者
- 十字鱼 https://space.bilibili.com/893892
- 更新日期 06.30
- 更新部分插件和模型
- 增加了下载检查，下载不成功无限重试

## 1.安装并运行Webui
- 如果启动不成功，请停止后重复运行第1步。

In [ ]:
import os
import time
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor
#安装目录
path = "/mnt/workspace"
#是否重装
reinstall = False
#是否双开
use2 = False
#SD模型（只放带文件名的直链）
model_urls = """
https://huggingface.co/Gluttony10/SD/resolve/main/chilloutmix-Ni.safetensors
https://huggingface.co/Gluttony10/SD/resolve/main/pinkFox30_pinkFox30_fp16.safetensors
https://huggingface.co/Gluttony10/SD/resolve/main/Anything-v5.0-PRT-RE.safetensors
https://huggingface.co/PWB/FeiWuMIX/resolve/main/FEIWUmix-2.5D.safetensors
https://huggingface.co/PWB/FeiWuMIX/resolve/main/FeiWuMIX-FlatCoating.safetensors
https://huggingface.co/PWB/FeiWuMIX/resolve/main/FeiWuMIX-v1-SlightFlat.safetensors
https://huggingface.co/chenxluo/QteaMix/resolve/main/QteaMix-%CE%A9.safetensors
https://huggingface.co/Meina/MeinaMix/resolve/main/Meina%20V10%20-%20baked%20VAE.safetensors
https://huggingface.co/luizC/sdhk/resolve/main/sdhk.safetensors
"""
#SD模型2（只放不带文件名的直链）
model2_urls = """
"""
#VAE模型（只放带文件名的直链）
VAE_urls = """
https://huggingface.co/Gluttony10/SD/resolve/main/vae-ft-mse-840000-ema-pruned.vae.ckpt
https://huggingface.co/swl-models/ClearVAE/resolve/main/ClearVAE-NansLessTest.safetensors
"""
#lora模型（只放带文件名的直链）
lora_urls = """
https://huggingface.co/Gluttony10/SD/resolve/main/lora.tar
https://huggingface.co/Gluttony10/SD/resolve/main/underboob.safetensors
"""
#ControlNet模型（只放带文件名的直链）
ControlNet_urls = """
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth
https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth
https://huggingface.co/ioclab/ioc-controlnet/resolve/main/models/control_v1p_sd15_brightness.safetensors
"""
#扩展插件
extension_urls = """
https://gitcode.net/ranting8323/a1111-sd-webui-tagcomplete
https://gitcode.net/overbill1683/stable-diffusion-webui-localization-zh_Hans
https://gitcode.net/ranting8323/sd-webui-additional-networks
https://gitcode.net/ranting8323/stable-diffusion-webui-wd14-tagger
https://gitcode.net/ranting8323/sd-webui-controlnet
https://ghproxy.com/https://github.com/fkunn1326/openpose-editor
https://ghproxy.com/https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper
https://gitcode.net/ranting8323/multidiffusion-upscaler-for-automatic1111
https://ghproxy.com/https://github.com/Akegarasu/sd-webui-video-frames
https://ghproxy.com/https://github.com/OedoSoldier/sd-webui-image-sequence-toolkit
https://ghproxy.com/https://github.com/AlUlkesh/stable-diffusion-webui-images-browser
https://ghproxy.com/https://github.com/Bing-su/adetailer
https://ghproxy.com/https://github.com/huchenlei/sd-webui-openpose-editor
https://ghproxy.com/https://github.com/nonnonstop/sd-webui-3d-open-pose-editor
"""
#
repositorie_urls = """
https://gitcode.net/overbill1683/taming-transformers
https://gitcode.net/overbill1683/k-diffusion
https://gitcode.net/overbill1683/CodeFormer
https://gitcode.net/overbill1683/BLIP
"""
#Deepdanbooru模型（只放带文件名的直链）
deepdanbooru_urls = """
https://ghproxy.com/https://github.com/AUTOMATIC1111/TorchDeepDanbooru/releases/download/v1/model-resnet_custom_v3.pt
"""
#ESRGAN模型（只放带文件名的直链）
ESRGAN_urls = """
https://ghproxy.com/https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth
"""
#Embedding模型（只放带文件名的直链）
embedding_urls = """
https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/badhandv4.pt
https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/easynegative.safetensors
https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/putnegative.safetensors
https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/verybadimagenegative_v1.3.pt
"""
#Adetailer模型（只放带文件名的直链）
adetailer_urls = """
https://huggingface.co/Bingsu/adetailer/resolve/main/deepfashion2_yolov8s-seg.pt
https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8n_v2.pt
"""
#额外文件（只放带文件名的直链）
extra_url = """
https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/train.sh
https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/sample_prompts.txt
"""
#安装webui（不要修改）
def install_webui():
    print('开始安装Webui')
    !git -C $path clone -q https://gitcode.net/overbill1683/stable-diffusion-webui > /dev/null 2>&1
    !pip install -q -U pip > /dev/null 2>&1
    !pip install -q -U torch torchvision torchaudio xformers > /dev/null 2>&1
    !pip install -q -U -r $path/stable-diffusion-webui/requirements.txt > /dev/null 2>&1
    !mkdir $path/stable-diffusion-webui/repositories > /dev/null 2>&1
    print('Webui安装完毕')
#安装lora（不要修改）
def install_lora():
    print('开始安装lora')
    !git -C $path clone -q https://gitcode.net/mirrors/kohya-ss/sd-scripts > /dev/null 2>&1
    print('lora安装完毕')
#内存优化（不要修改）
def libtcmalloc():
    print('开始安装内存优化')
    !apt-get update -qq
    !apt-get install -qq -y aria2
    !git config --global http.postBuffer 2000000000
    if os.path.exists(f'{path}/temp'):
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        print('内存优化已安装')
    else:
        os.chdir(f'{path}')
        os.makedirs('temp', exist_ok=True)
        os.chdir('temp')
        os.system('wget -q http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb')
        os.system('apt-get install -qq libunwind8-dev -y > /dev/null 2>&1')
        !dpkg -i *.deb
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        !rm *.deb    
        print('内存优化安装完毕')
#安装webui2（不要修改）
def install_webui2():
    !git -C $path/stable-diffusion-webui pull -q
    !wget -q -O $path/stable-diffusion-webui/config.json https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/config.json
    !wget -q -O $path/stable-diffusion-webui/styles.csv https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/styles.csv
#安装lora2（不要修改）
def install_lora2():
    !git -C $path/sd-scripts pull -q
    %cd $path/sd-scripts
    !pip install -q -U -r $path/sd-scripts/requirements.txt > /dev/null 2>&1
    !pip install -q -U lion-pytorch dadaptation lycoris-lora fastapi uvicorn wandb Send2Trash mediapipe fvcore > /dev/null 2>&1
    !mkdir $path/train > /dev/null 2>&1
    !mkdir $path/train/10_glut > /dev/null 2>&1
#下载文件（不要修改）
def download_all():
    print()
    tasks = []
    tasks.append(f'git -C {path}/stable-diffusion-webui/repositories clone -q https://gitcode.net/overbill1683/stablediffusion stable-diffusion-stability-ai > /dev/null 2>&1')
    tasks.append(f'aria2c --console-log-level=error -q -c -d {path}/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/ -o downloads.tar https://huggingface.co/Gluttony10/SD/resolve/main/downloads.tar > /dev/null 2>&1')
    tasks.extend(download_git(repositorie_urls,f'{path}/stable-diffusion-webui/repositories'))
    tasks.extend(download_aria(model_urls,f'{path}/stable-diffusion-webui/models/Stable-diffusion'))
    tasks.extend(download_aria2(model2_urls,f'{path}/stable-diffusion-webui/models/Stable-diffusion'))
    tasks.extend(download_aria(VAE_urls,f'{path}/stable-diffusion-webui/models/VAE'))
    tasks.extend(download_aria(lora_urls,f'{path}/stable-diffusion-webui/models/Lora'))
    tasks.extend(download_aria(ControlNet_urls,f'{path}/stable-diffusion-webui/models/ControlNet'))
    tasks.extend(download_aria(deepdanbooru_urls,f'{path}/stable-diffusion-webui/models/torch_deepdanbooru'))
    tasks.extend(download_aria(ESRGAN_urls,f'{path}/stable-diffusion-webui/models/RealESRGAN'))
    tasks.extend(download_git(extension_urls,f'{path}/stable-diffusion-webui/extensions'))
    tasks.extend(download_aria(embedding_urls,f'{path}/stable-diffusion-webui/embeddings'))
    tasks.extend(download_aria(adetailer_urls,f'{path}/stable-diffusion-webui/models/adetailer'))
    tasks.extend(download_aria(extra_url,f'{path}'))
    with ThreadPoolExecutor() as executor:
        for task in tasks:
            executor.submit(os.system, task)
#下载git库（不要修改）
def download_git(links,folder):
    tasks = []
    link = links.strip().split("\n")
    for li in link:
        tasks.append(f'git -C {folder} clone -q {li} > /dev/null 2>&1')
        name = li.split('/')[-1]
        tasks.append(f'git -C {folder}/{name} pull -q > /dev/null 2>&1')
    return tasks
#下载模型（不要修改）
def download_aria(links,folder):
    global check
    tasks = []
    link = links.strip().split("\n")
    for li in link:
        name = li.split('/')[-1]
        tasks.append(f'aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {folder} -o {name} "{li}" > /dev/null 2>&1')
        check = check & os.path.exists(f'{folder}/{name}')
    return tasks
#下载模型2（不要修改）
def download_aria2(links,folder):
    tasks = []
    link = links.strip().split("\n")
    for li in link:
        tasks.append(f'aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d {folder} "{li}" > /dev/null 2>&1')
    return tasks
#显卡0（不要修改）
def start_1():
    %cd $path/stable-diffusion-webui
    !python launch.py --no-download-sd-model --xformers --share --listen --enable-insecure-extension-access
#显卡1（不要修改）
def start_2(): 
    if use2:
        %cd $path/stable-diffusion-webui
        !python launch.py --lowvram --no-download-sd-model --xformers --share --listen --enable-insecure-extension-access --port 7861 --device-id=1
    pass
#主进程（不要修改）
def main():
    starttime = time.time()
    print("运行开始")
    !nvidia-smi #查看显卡
    %cd $path
    if reinstall:
        print('旧文件删除中')
        !rm -rf stable-diffusion-webui
        !rm -rf sd-scripts
        !rm -rf temp
    with ProcessPoolExecutor() as executor:
        futures = []
        for func in [install_webui,install_lora,libtcmalloc]:
            futures.append(executor.submit(func))
            time.sleep(1)
        for future in futures:
            future.result()
    if os.path.exists(f'{path}/stable-diffusion-webui')&os.path.exists(f'{path}/sd-scripts'):
        print('\033[32mwebui、lora和内存优化安装完毕，等待下载模型\033[0m')
        with ProcessPoolExecutor() as executor:
            futures = []
            for func in [install_webui2,install_lora2]:
                futures.append(executor.submit(func))
                time.sleep(1)
            for future in futures:
                future.result()
        !tar -xf $path/stable-diffusion-webui/models/Lora/lora.tar -C $path/stable-diffusion-webui/models/Lora/
        !tar -xf $path/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/downloads.tar -C $path/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/
        download_all()
        endtime = time.time()
        print('\033[32m安装耗时:',int((endtime - starttime)/60),'min\033[0m')
        if check:
            print('\033[32m模型下载完毕\033[0m')
        else:
            print('\033[31m模型下载失败，将开启无限重试\033[0m')
        with ProcessPoolExecutor() as executor:
            futures = []
            for func in [start_1,start_2]:
                futures.append(executor.submit(func))
                time.sleep(61)
            while not check:
                time.sleep(60)
                futures.append(executor.submit(download_all))
            for future in futures:
                future.result()
    else:
        print('安装失败请重试')

check = True
if __name__ == '__main__':
    main()

## 2.压缩outputs文件夹
- 运行前先停止步骤1

In [ ]:
%cd $path/stable-diffusion-webui
!tar -zcvf 1.tar.gz outputs
!mv 1.tar.gz ..
!rm -rf outputs

## 3.lora训练
- 运行前先停止步骤1
- 训练文件放进./train/10_glut文件夹中
- 训练参数请修改 ./train.sh

In [ ]:
%cd $path
!bash train.sh